<a href="https://colab.research.google.com/github/GabrielDeml/TumorClassifier/blob/main/TumorClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %tensorflow_version 1.x

In [2]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 3.9MB/s 
     |████████████████████████████████| 112kB 7.9MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models

import IPython.display as display
import pathlib
import tensorflow as tf
import os
# tf.enable_eager_execution()
tf.__version__

import tensorflowjs as tfjs

In [4]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [5]:
!mkdir .kaggle

In [6]:
import json
token = {"username":"gabrieldeml","key":"3e37c76c042a65523596d5fdd5b719ae"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [7]:
!chmod 600 /content/.kaggle/kaggle.json

In [8]:
mkdir ~/.kaggle/

In [9]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [10]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [11]:
#!kaggle competitions download -c dogs-vs-cats -p /content
!kaggle datasets download -d fanconic/skin-cancer-malignant-vs-benign -p /content

 96% 313M/325M [00:10<00:00, 27.7MB/s]
100% 325M/325M [00:10<00:00, 32.5MB/s]


In [12]:
print ("skin-cancer-malignant-vs-benign: {}".format(os.path.exists("skin-cancer-malignant-vs-benign.zip")))
if not (os.path.exists("skin-cancer-malignant-vs-benign")):
  !unzip -q skin-cancer-malignant-vs-benign.zip
else:
  print("We already did this")

skin-cancer-malignant-vs-benign: True


In [13]:
!mkdir train/cat
!mkdir train/dog
!mv train/cat* train/cat
!mv train/dog* train/dog

mv: cannot move 'train/cat' to a subdirectory of itself, 'train/cat/cat'
mv: cannot move 'train/dog' to a subdirectory of itself, 'train/dog/dog'


In [14]:
data_root = pathlib.Path("data/train")
data_test = pathlib.Path("data/test")

In [15]:
for item in data_root.iterdir():
  print(item)

data/train/benign
data/train/malignant


In [16]:
# TODO Add the rest of me for test
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

2637

In [17]:
import random
all_image_paths_test = list(data_test.glob('*/*'))
all_image_paths_test = [str(path) for path in all_image_paths_test]

image_count_test = len(all_image_paths_test)
image_count_test

660

In [18]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['benign', 'malignant']

In [19]:
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

{'benign': 0, 'malignant': 1}

In [20]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])

First 10 labels indices:  [1, 0, 0, 1, 0, 0, 0, 1, 0, 0]


In [21]:
all_image_labels_test = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths_test]

print("First 10 labels indices: ", all_image_labels_test[:10])

First 10 labels indices:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [22]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [224, 224])
  image /= 255.0  # normalize to [0,1] range

  return image

In [23]:
def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [24]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))
ds_test = tf.data.Dataset.from_tensor_slices((all_image_paths_test, all_image_labels_test))
# The tuples are unpacked into the positional arguments of the mapped function
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
ds_test = ds_test.map(load_and_preprocess_from_path_label)
image_label_ds 
ds_test

<MapDataset shapes: ((224, 224, 3), ()), types: (tf.float32, tf.int32)>

In [25]:
ds = image_label_ds.apply(
tf.data.experimental.shuffle_and_repeat(buffer_size=500))
ds = ds.batch(32)
ds = ds.prefetch(buffer_size=5000)
ds = ds.repeat()
ds

ds_test = ds_test.batch(32)
ds_test = ds_test.prefetch(buffer_size=500)
# ds = image_label_ds.apply(
#   tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
# ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
# ds
# ds = ds.shuffle(buffer_size=10000) ## 10000: size of sample/record pool for random selection
# ds = ds.batch(32) ## 32: number of samples/records per batch (to be read into memory)
# ds = ds.repeat() ## None: keep repeating

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [26]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [27]:
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 173056)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [29]:
model.fit(ds, epochs=600, steps_per_epoch=3, )

Epoch 1/600
3/3 [==============================] - 32s 214ms/step - loss: 3.6671 - accuracy: 0.5013
Epoch 2/600
3/3 [==============================] - 0s 31ms/step - loss: 5.6912 - accuracy: 0.4883
Epoch 3/600
3/3 [==============================] - 0s 35ms/step - loss: 0.8826 - accuracy: 0.4297
Epoch 4/600
3/3 [==============================] - 0s 33ms/step - loss: 0.8208 - accuracy: 0.4375
Epoch 5/600
3/3 [==============================] - 0s 34ms/step - loss: 0.7408 - accuracy: 0.5378
Epoch 6/600
3/3 [==============================] - 0s 34ms/step - loss: 0.6578 - accuracy: 0.6250
Epoch 7/600
3/3 [==============================] - 0s 32ms/step - loss: 0.6071 - accuracy: 0.7083
Epoch 8/600
3/3 [==============================] - 0s 32ms/step - loss: 0.6176 - accuracy: 0.6927
Epoch 9/600
3/3 [==============================] - 0s 32ms/step - loss: 0.5301 - accuracy: 0.7539
Epoch 10/600
3/3 [==============================] - 0s 32ms/step - loss: 0.6386 - accuracy: 0.6432
Epoch 11/600
3/3 

In [36]:
print("Evaluate")
result = model.evaluate(ds_test)
dict(zip(model.metrics_names, result))


Evaluate
21/21 [==============================] - 1s 50ms/step - loss: 1.1632 - accuracy: 0.8091


{'accuracy': 0.8090909123420715, 'loss': 1.1632131338119507}

In [31]:
tfjs.converters.save_keras_model(model, "saved_model")


/usr/local/lib/python3.7/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:123: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [32]:
!zip -r saved_model.zip saved_model/

  adding: saved_model/ (stored 0%)
  adding: saved_model/group1-shard11of11.bin (deflated 7%)
  adding: saved_model/group1-shard9of11.bin (deflated 7%)
  adding: saved_model/group1-shard6of11.bin (deflated 7%)
  adding: saved_model/group1-shard2of11.bin (deflated 7%)
  adding: saved_model/group1-shard4of11.bin (deflated 7%)
  adding: saved_model/group1-shard3of11.bin (deflated 7%)
  adding: saved_model/group1-shard8of11.bin (deflated 7%)
  adding: saved_model/group1-shard10of11.bin (deflated 7%)
  adding: saved_model/group1-shard1of11.bin (deflated 7%)
  adding: saved_model/group1-shard7of11.bin (deflated 7%)
  adding: saved_model/model.json (deflated 82%)
  adding: saved_model/group1-shard5of11.bin (deflated 7%)


In [33]:
# !mkdir -p saved_model
# model.save('saved_model/model')

In [34]:
# # Convert the model
# converter = tf.lite.TFLiteConverter.from_saved_model('/saved_model/model.pb') 
# # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#   f.write(tflite_model)


In [35]:
# # Convert the model.
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#   f.write(tflite_model)
